In [36]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interact
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm, ListedColormap
from IPython.display import display
import math
%matplotlib inline

# from gnc.control import PIDController, Trajectory, getDistance


In [47]:

Kp = 1.0
Ki = 0.0
Kd = 0.0

dt = 0.5 # [s]
T = 60 # [s]

In [14]:
def normalize(num, lower=0.0, upper=360.0, b=False):
    """Normalize number to range [lower, upper) or [lower, upper].
    Parameters
    ----------
    num : float
        The number to be normalized.
    lower : float
        Lower limit of range. Default is 0.0.
    upper : float
        Upper limit of range. Default is 360.0.
    b : bool
        Type of normalization. See notes.
    Returns
    -------
    n : float
        A number in the range [lower, upper) or [lower, upper].
    Raises
    ------
    ValueError
      If lower >= upper.
    Notes
    -----
    If the keyword `b == False`, the default, then the normalization
    is done in the following way. Consider the numbers to be arranged
    in a circle, with the lower and upper marks sitting on top of each
    other. Moving past one limit, takes the number into the beginning
    of the other end. For example, if range is [0 - 360), then 361
    becomes 1. Negative numbers move from higher to lower
    numbers. So, -1 normalized to [0 - 360) becomes 359.
    If the keyword `b == True` then the given number is considered to
    "bounce" between the two limits. So, -91 normalized to [-90, 90],
    becomes -89, instead of 89. In this case the range is [lower,
    upper]. This code is based on the function `fmt_delta` of `TPM`.
    Range must be symmetric about 0 or lower == 0.
    Examples
    --------
    >>> normalize(-270,-180,180)
    90
    >>> import math
    >>> math.degrees(normalize(-2*math.pi,-math.pi,math.pi))
    0.0
    >>> normalize(181,-180,180)
    -179
    >>> normalize(-180,0,360)
    180
    >>> normalize(36,0,24)
    12
    >>> normalize(368.5,-180,180)
    8.5
    >>> normalize(-100, -90, 90, b=True)
    -80.0
    >>> normalize(100, -90, 90, b=True)
    80.0
    >>> normalize(181, -90, 90, b=True)
    -1.0
    >>> normalize(270, -90, 90, b=True)
    -90.0
    """
    from math import floor, ceil
    # abs(num + upper) and abs(num - lower) are needed, instead of
    # abs(num), since the lower and upper limits need not be 0. We need
    # to add half size of the range, so that the final result is lower +
    # <value> or upper - <value>, respectively.
    res = num
    if not b:
        if lower >= upper:
            raise ValueError("Invalid lower and upper limits: (%s, %s)" %
                             (lower, upper))

        res = num
        if num > upper or num == lower:
            num = lower + abs(num + upper) % (abs(lower) + abs(upper))
        if num < lower or num == upper:
            num = upper - abs(num - lower) % (abs(lower) + abs(upper))

        res = lower if res == upper else num
    else:
        total_length = abs(lower) + abs(upper)
        if num < -total_length:
            num += ceil(num / (-2 * total_length)) * 2 * total_length
        if num > total_length:
            num -= floor(num / (2 * total_length)) * 2 * total_length
        if num > upper:
            num = total_length - num
        if num < lower:
            num = -total_length - num

        res = num * 1.0  # Make all numbers float, to be consistent

    return res


def d2d(d):
    """Normalize angle in degree to [0, 360)."""
    return normalize(d, 0, 360)


def h2h(h):
    """Normalize angle in hours to [0, 24.0)."""
    return normalize(h, 0, 24)


def r2r(r):
    """Normalize angle in radians to [0, 2π)."""
    return normalize(r, 0, 2 * math.pi)


In [15]:
class PIDController:
	def __init__(self, Kp, Ki, Kd, offset=0):
		'''
		Initialize the PID controller.

		Args:
			Kp (float): Proportional gain
			Ki (float): Integral gain
			Kd (float): Derivative gain
		'''
		self.Kp = Kp
		self.Ki = Ki
		self.Kd = Kd
		self.prev_error = 0
		self.integral = 0
		self.offset = offset
		# self.prev_time = datetime.now()

	def calculate(self, setpoint, feedback_value):
		'''
		Calculate the PID control output.

		Args:
			setpoint (float): The desired value
			feedback_value (float): The actual value

		Returns:
			float: The PID control output
		'''
		error = setpoint - feedback_value
		# dt = 0.01

		P = self.Kp * error
		self.integral = self.integral + self.Ki * error * dt
		D = self.Kd * (error - self.prev_error) / dt

		output = self.offset + self.integral + P + D

		self.prev_error = error

		return output


def getDistance(p1, p2):
	"""
	Calculate distance
	:param p1: list, point1
	:param p2: list, point2
	:return: float, distance
	"""
	dx = p1[0] - p2[0]
	dy = p1[1] - p2[1]
	return math.hypot(dx, dy)


class Trajectory:
	def __init__(self, routePoints: list[tuple[int, int]], L = 5):
		"""
		Define a trajectory class
		:param routePoints: list, list of positions
		:param L: int, look ahead distance
		"""
		traj_x, traj_y = zip(*routePoints)
		self.traj_x = traj_x
		self.traj_y = traj_y
		self.last_idx = 0
		self.L = L

	def getPoint(self, idx) -> list[int]:
		return [self.traj_x[idx], self.traj_y[idx]]

	def getTargetPoint(self, pos) -> list[int]:
		"""
		Get the next look ahead point
		:param pos: list, vehicle position
		:return: list, target point
		"""
		target_idx = self.last_idx
		target_point = self.getPoint(target_idx)
		curr_dist = getDistance(pos, target_point)

		while curr_dist < self.L and target_idx < len(self.traj_x) - 1:
			target_idx += 1
			target_point = self.getPoint(target_idx)
			curr_dist = getDistance(pos, target_point)

		self.last_idx = target_idx
		return self.getPoint(target_idx)




In [63]:
class MotorSimNew():
    def __init__(self) -> None:
        self.st = 200
        self.w = 0
        self.v = 0
        self.speed = 0
        self.r = 0.15 / 2  #[m]

    def steps(self):
        return self.st
    
    # Motor proto
    def setAngularVelocity(self, w):
        '''units: rad/s'''
        self.w = w
        self.v = w * self.r
        self.calcSpeed()

    def calcSpeed(self):
        self.speed = (self.w * self.st) / (math.pi * 2 * self.r)
    
    def angularVelocity(self):
        '''units: rad/s'''
        return self.w
    
    def rps(self):
        return self.speed / self.st

    # Wheel proto
    def wheelRadius(self):
        '''units: m'''
        return self.r
    
    def setLinearVelocity(self, v):
        '''units: m/s'''
        self.v = v
        self.w = v / self.r
        self.calcSpeed()

    def linearVelocity(self):
        '''units: m/s'''
        return self.v


class WheelbaseNew():
    def __init__(self) -> None:
        self.theta = 0
        self.w = 0      # [deg/s]
        self.v = 0      # [m/s]
        self.B = 0.7    # [m]

        self.lm = MotorSimNew()
        self.rm = MotorSimNew()

    def wheelDistance(self):
        '''units: m'''
        return self.B
    
    def linearVelocity(self):
        '''units: m/s'''
        return self.v #(self.rm.v + self.lm.v) / 2
    
    def angularVelocity(self):
        '''units: deg/s'''
        return self.w #(self.rm.v + self.lm.v) / self.B
    
    def setAngularVelocity(self, w):
        '''units: deg/s'''
        self.w = w
        self.updateMotors()
        self.v = self.calcLinearVelocity()

    def setLinearVelocity(self, v):
        '''units: m/s'''
        self.v = v
        self.updateMotors()
        self.w = self.calcAngularVelocity()
    
    def updateMotors(self):
        wL = (self.v / self.lm.r) - (self.B * self.w) / (2 * self.lm.r)
        wR = (self.v / self.rm.r) + (self.B * self.w) / (2 * self.rm.r)
        self.lm.setAngularVelocity(wL)
        self.rm.setAngularVelocity(wR)

    def calcAngularVelocity(self):
        self.w = (self.rm.r / self.B) * (self.rm.w - self.lm.w)
        return self.w
    
    def calcLinearVelocity(self):
        self.v = (self.rm.r / 2) * (self.rm.w + self.lm.w)
        return self.v
    

class FaithNew():
    def __init__(self) -> None:
        self.wb = WheelbaseNew()
        self.h = 0
        self.pos = np.array([45.30, 2.60, 0.0])
        self.vel = np.array([0.0, 0.0, 0.0])

    def update(self):
        delta = self.wb.angularVelocity() * dt
        self.h += delta
        normalize(self.h, 0, 360)
        theta = math.radians(self.h)
        vx = self.wb.calcLinearVelocity() * math.cos(theta)
        vy = self.wb.calcLinearVelocity() * math.sin(theta)
        self.pos[0] += vx * dt; self.pos[1] += vy * dt
        self.vel[0] = vx; self.vel[1] = vy


In [64]:
class MotorSim():
    def __init__(self):
        self._speed = 0
        self.dir = 0
        self._steps = None
        if self._steps is None:
            self._steps = 200
        self.maxSpeed = 1200
        self.accel = None
        if self.accel is None:
            self.accel = 500
        self.channel = 0

    def steps(self):
        return self._steps

    def setSpeed(self, speed: int):
        # print(f"Set speed {speed}")
        if speed > self.maxSpeed: speed = self.maxSpeed
        self._speed = int(speed)

    def stop(self):
        self.setSpeed(0)

    def enable(self):
        ...

    def disable(self):
        ...

    def speed(self):
        return self._speed


class Wheelbase():

    def __init__(self):
        self._angle = 0 # Forward, north
        self._motorEnabled = True
        self._proximityCheckEnabled = True
        self.speed = 0
        self.maxSpeed = 0
        self.x = 0
        self.y = 0
        self.r = 0
        self.lm = MotorSim()
        self.rm = MotorSim()

    def setAngle(self, radians: float):
        _angle = radians
        # xa, ya, wx, wy, RM, LM, RMx, RMy, LMx, LMy: float
        xa = math.cos(_angle)
        ya = math.sin(_angle)
        RMx = xa
        LMx = xa
        RMy = -ya
        LMy = ya
        th = 0
        nu = 0
        if ya > 0:
            if xa > 0:
                # TOP RIGHT
                th = +1
                nu = +1
            else:
                # TOP LEFT
                th = -1
                nu = -1
        else:
            if xa < 0:
                # BOTTOM LEFT
                th = -1
                nu = +1
            else:
                # BOTTOM RIGHT
                th = +1
                nu = -1
        wx = math.fabs((1+(xa*th-ya*nu))/2)
        wy = math.fabs((1-(xa*th-ya*nu))/2)
        RM = RMx * wx + RMy * wy
        LM = LMx * wx + LMy * wy

        speedR = self.maxSpeed * RM * self.r
        speedL = self.maxSpeed * LM * self.r
        # print((RM, self.maxSpeed, self.r, speedR))
        # _logger.debug(f'speedL, speedR:, {speedL}, {speedR}, LM, RM:, {LM}, {RM}')
        self.work(speedR, speedL)

    def setAngularVel(self, w: float):
        '''w is deg/s'''
        r = self.diameter() / 2
        b = self.distance()
        V = self.maxSpeed
        stepsR = self.rm.steps() or 200
        stepsL = self.lm.steps() or 200

        # w = normalize(w, -math.pi, math.pi)
        # w = normalize(math.degrees(w), -180, 180)
        w = normalize(w, -180, 180)
        # w = normalize(w, -180, 180)
        # w = math.degrees(w)
        # w = math.radians(w)

        wR = (V + w*(b/2))/r
        wL = (V - w*(b/2))/r

        # ww = np.pi * r*2 * (v / steps)
        speedR = wR * stepsR / np.pi * r*2
        speedL = wL * stepsL / np.pi * r*2

        self.work(speedR, speedL)
        # self.work(-speedR, -speedL)

    def work(self, speedR, speedL):
        self.rm.setSpeed(speedR)
        self.lm.setSpeed(speedL)

    # @log_on_start(logging.INFO, "Wheels ON")
    def enable(self, force = False):
        ...
        # if not self._motorEnabled or force:
        #     self.lm.enable()
        #     self.rm.enable()
        #     self._motorEnabled = True

    # @log_on_start(logging.INFO, "Wheels OFF")
    def disable(self, force = False):
        ...
        # if self._motorEnabled or force:
        #     self.lm.disable()
        #     self.rm.disable()
        #     self._motorEnabled = False

    def inputXY(self, x, y):
        # x = x
        # y = y
        angle = math.atan2(y, x)
        self.r = math.hypot(x, y)
        # _logger.debug(f"r: {r}")
        if self.r < 0.10 and self.r > -0.10:
            self.r = 0
        # print("joystick R:", r)
        # print("angle:", angle)
        # self.setSpeed(maxSpeed * r)
        self.setAngle(angle) #math.radians(int(math.degrees(angle)+180)%360))


    def setSpeed(self, speed: float):
        '''speed in range [-1, 1]'''
        if speed < -1: speed = -1
        if speed > 1: speed = 1
        self.r = speed

    # def speedToSteps(self, speedMS):
    #     d = 0.075 # metres
    #     P = d*math.pi
    #     c1 = 1 / P
    #     n = P * c1 * 360


    def setMaxSpeed(self, maxSpeed):
        self.maxSpeed = maxSpeed
        # print(f"Maximum speed is set to {maxSpeed}")

    def maximumSpeed(self):
        return self.maxSpeed

    def angle(self):
        return math.degrees(self._angle)

    def finish(self):
        self.disable(force = True)

    def isEnabled(self):
        return self._motorEnabled

    def setProximityChecks(self, enabled: bool):
        self._proximityCheckEnabled = enabled

    def proximityChecks(self) -> bool:
        return self._proximityCheckEnabled

    def diameter(self): return 0.15

    def distance(self): return 0.7


class Faith():

    def __init__(self):
        self.wb = Wheelbase()
        self.mL = self.wb.lm
        self.mR = self.wb.rm

        self.heading = 180
        self.wheelD = self.wb.diameter()
        self.wheelDistance = self.wb.distance()
        self.lastPosition = np.array([453.0, 26.0, 0.0])
        self.velocity = np.array([0.0, 0.0, 0.0])
        self.linvelocity = 0

    def update(self):
        speed = np.array([self.mL.speed(), self.mR.speed()])
        steps = np.mean(np.array([self.mL.steps(), self.mR.steps()]))
        rps = speed / steps
        rate = np.pi * self.wheelD * rps # [rad/s]
        w = (rate[1] - rate[0]) / self.wheelDistance # [deg/s]  # do not mix up left and right
        deltaHeading = w * dt
        # normalize(deltaHeading, -180, 180)
        
        # deltaHeading = math.degrees(w) * dt
        # deltaHeading = math.radians(w) * dt
         # [deg]
        # print(self.heading)
        # self.heading += deltaHeading
        # normalize(self.heading, -math.pi, math.pi)
        # normalize(self.heading, -180, 180)
        
        
        self.velocity = np.array([np.cos(math.radians(self.heading)), np.sin(math.radians(self.heading))]) * rate # [m/s]
        # self.velocity = np.array([np.cos((self.heading)), np.sin((self.heading))]) * rate
        self.velocity = np.append(self.velocity, [0])
        # self.linvelocity = (rate[0] + rate[1])*(self.wheelD/2) / 2
        self.lastPosition = self.lastPosition + self.velocity * dt
        # self.lastPosition[0] += (self.linvelocity*dt) * math.cos(math.degrees(self.heading)+deltaHeading/2)
        # self.lastPosition[1] += (self.linvelocity*dt) * math.sin(math.degrees(self.heading)+deltaHeading/2)
        self.heading += deltaHeading
        # normalize(self.heading, -180, 180)
        

In [65]:
# routePoints = [
#     [450.0, 20.0], [440.0, 30.0], [440.0, 40.0], [440.0, 50.0], [430.0, 60.0], [420.0, 60.0], [410.0, 60.0], [400.0, 60.0], [390.0, 60.0], [380.0, 60.0], [370.0, 60.0], [360.0, 60.0], [350.0, 60.0], [340.0, 60.0], [330.0, 60.0], [320.0, 60.0], [310.0, 70.0], [300.0, 70.0], [290.0, 70.0], [280.0, 70.0], [270.0, 70.0], [260.0, 70.0], [250.0, 70.0], [240.0, 70.0], [230.0, 70.0], [220.0, 70.0], [210.0, 70.0], [200.0, 70.0], [190.0, 70.0], [180.0, 70.0], [170.0, 70.0], [160.0, 70.0], [150.0, 70.0], [140.0, 70.0], [130.0, 70.0], [120.0, 70.0], [110.0, 70.0], [100.0, 70.0], [90.0, 70.0], [80.0, 70.0]
# ]

routePoints = [
    [450.0, 20.0], [450.0, 30.0], [450.0, 40.0], [460.0, 50.0], [470.0, 60.0], [480.0, 70.0], [490.0, 80.0], [500.0, 80.0], [510.0, 80.0], [520.0, 80.0], [530.0, 80.0], [540.0, 80.0], [550.0, 80.0], [560.0, 80.0], [570.0, 80.0], [580.0, 80.0], [590.0, 80.0], [600.0, 80.0], [610.0, 80.0], [620.0, 80.0], [630.0, 80.0], [640.0, 80.0], [650.0, 80.0], [660.0, 80.0], [670.0, 80.0], [680.0, 80.0], [690.0, 80.0], [700.0, 80.0], [710.0, 80.0], [720.0, 90.0], [720.0, 100.0], [720.0, 110.0], [720.0, 120.0], [720.0, 130.0], [720.0, 140.0], [710.0, 150.0], [700.0, 160.0], [690.0, 170.0], [680.0, 170.0], [670.0, 170.0], [660.0, 170.0], [650.0, 180.0], [640.0, 190.0], [630.0, 200.0], [630.0, 210.0], [630.0, 220.0], [630.0, 230.0], [630.0, 240.0], [620.0, 250.0], [610.0, 250.0], [600.0, 250.0], [590.0, 250.0], [580.0, 250.0], [570.0, 250.0], [560.0, 250.0], [550.0, 240.0], [540.0, 230.0], [530.0, 220.0], [530.0, 210.0], [530.0, 200.0], [530.0, 190.0], [530.0, 180.0], [520.0, 170.0], [510.0, 170.0], [500.0, 170.0], [490.0, 170.0], [480.0, 170.0], [470.0, 170.0], [460.0, 180.0], [460.0, 190.0], [460.0, 200.0], [460.0, 210.0], [460.0, 220.0], [460.0, 230.0], [460.0, 240.0], [460.0, 250.0], [460.0, 260.0], [460.0, 270.0], [460.0, 280.0], [460.0, 290.0], [460.0, 300.0], [460.0, 310.0], [460.0, 320.0], [460.0, 330.0], [470.0, 340.0], [480.0, 340.0], [490.0, 340.0], [500.0, 340.0], [510.0, 340.0], [520.0, 340.0], [530.0, 340.0], [540.0, 340.0], [550.0, 340.0], [560.0, 340.0], [570.0, 340.0], [580.0, 340.0], [590.0, 340.0], [600.0, 340.0], [610.0, 350.0], [620.0, 360.0], [630.0, 370.0], [630.0, 380.0], [630.0, 390.0], [630.0, 400.0], [630.0, 410.0], [630.0, 420.0], [630.0, 430.0], [620.0, 440.0], [610.0, 440.0], [600.0, 440.0], [590.0, 440.0], [580.0, 440.0], [570.0, 450.0], [560.0, 460.0], [550.0, 470.0], [550.0, 480.0], [550.0, 490.0], [550.0, 500.0], [550.0, 510.0], [550.0, 520.0], [550.0, 530.0], [550.0, 540.0], [550.0, 550.0], [550.0, 560.0], [550.0, 570.0], [550.0, 580.0], [550.0, 590.0], [550.0, 600.0], [550.0, 610.0], [550.0, 620.0], [560.0, 630.0], [570.0, 630.0], [580.0, 630.0], [590.0, 630.0], [600.0, 630.0], [610.0, 630.0], [620.0, 630.0], [630.0, 630.0], [640.0, 630.0], [650.0, 630.0], [660.0, 630.0], [670.0, 630.0], [680.0, 630.0], [690.0, 630.0], [700.0, 640.0], [710.0, 650.0], [720.0, 660.0], [720.0, 670.0], [720.0, 680.0], [720.0, 690.0], [730.0, 700.0], [740.0, 710.0], [750.0, 720.0], [760.0, 720.0], [770.0, 720.0], [780.0, 720.0], [790.0, 720.0], [800.0, 720.0], [810.0, 720.0], [820.0, 720.0], [830.0, 720.0], [840.0, 720.0], [850.0, 720.0], [860.0, 720.0], [870.0, 720.0], [880.0, 720.0], [890.0, 730.0], [900.0, 740.0], [910.0, 750.0], [910.0, 760.0], [910.0, 770.0], [910.0, 780.0], [910.0, 790.0], [910.0, 800.0], [910.0, 810.0], [910.0, 820.0], [910.0, 830.0], [910.0, 840.0], [910.0, 850.0], [910.0, 860.0], [910.0, 870.0], [910.0, 880.0], [910.0, 890.0], [910.0, 900.0], [900.0, 910.0], [890.0, 910.0], [880.0, 910.0], [870.0, 910.0], [860.0, 910.0], [850.0, 910.0], [840.0, 910.0], [830.0, 910.0], [820.0, 910.0], [810.0, 910.0], [800.0, 910.0], [790.0, 910.0], [780.0, 910.0], [770.0, 910.0], [760.0, 910.0], [750.0, 910.0], [740.0, 910.0], [730.0, 910.0], [720.0, 910.0], [710.0, 910.0], [700.0, 910.0], [690.0, 910.0], [680.0, 910.0], [670.0, 910.0], [660.0, 910.0], [650.0, 910.0], [640.0, 910.0], [630.0, 910.0], [620.0, 910.0], [610.0, 910.0], [600.0, 910.0], [590.0, 910.0], [580.0, 910.0], [570.0, 910.0], [560.0, 910.0], [550.0, 900.0], [550.0, 890.0], [550.0, 880.0], [550.0, 870.0], [540.0, 860.0], [530.0, 850.0], [520.0, 840.0], [510.0, 830.0], [500.0, 820.0], [490.0, 820.0], [480.0, 820.0], [470.0, 820.0], [460.0, 820.0], [450.0, 820.0], [440.0, 820.0], [430.0, 820.0], [420.0, 820.0], [410.0, 820.0], [400.0, 820.0], [390.0, 820.0], [380.0, 820.0], [370.0, 820.0], [360.0, 810.0], [360.0, 800.0], [360.0, 790.0], [360.0, 780.0], [360.0, 770.0], [350.0, 760.0], [340.0, 750.0], [330.0, 740.0], [320.0, 740.0], [310.0, 740.0], [300.0, 740.0], [290.0, 740.0], [280.0, 740.0], [270.0, 740.0], [260.0, 740.0], [250.0, 740.0], [240.0, 740.0], [230.0, 740.0], [220.0, 740.0], [210.0, 740.0], [200.0, 740.0], [190.0, 740.0], [180.0, 740.0], [170.0, 750.0], [170.0, 760.0], [170.0, 770.0], [170.0, 780.0], [170.0, 790.0], [160.0, 800.0], [150.0, 810.0], [140.0, 820.0], [130.0, 820.0], [120.0, 820.0], [110.0, 820.0], [100.0, 820.0], [90.0, 820.0], [80.0, 810.0], [80.0, 800.0], [80.0, 790.0], [80.0, 780.0], [80.0, 770.0], [80.0, 760.0], [80.0, 750.0], [80.0, 740.0], [80.0, 730.0], [80.0, 720.0], [80.0, 710.0], [80.0, 700.0], [80.0, 690.0], [80.0, 680.0], [80.0, 670.0], [80.0, 660.0], [80.0, 650.0], [80.0, 640.0], [80.0, 630.0], [80.0, 620.0], [80.0, 610.0], [80.0, 600.0], [80.0, 590.0], [80.0, 580.0], [80.0, 570.0], [80.0, 560.0], [80.0, 550.0], [80.0, 540.0], [80.0, 530.0], [80.0, 520.0], [80.0, 510.0], [80.0, 500.0], [80.0, 490.0], [80.0, 480.0], [80.0, 470.0], [80.0, 460.0], [80.0, 450.0], [80.0, 440.0], [80.0, 430.0], [80.0, 420.0], [80.0, 410.0], [80.0, 400.0], [80.0, 390.0], [80.0, 380.0], [80.0, 370.0], [80.0, 360.0], [80.0, 350.0], [80.0, 340.0], [80.0, 330.0], [80.0, 320.0], [80.0, 310.0], [80.0, 300.0], [80.0, 290.0], [80.0, 280.0], [80.0, 270.0], [80.0, 260.0], [80.0, 250.0], [80.0, 240.0], [80.0, 230.0], [80.0, 220.0], [80.0, 210.0], [80.0, 200.0], [80.0, 190.0], [80.0, 180.0], [90.0, 170.0], [100.0, 170.0], [110.0, 170.0], [120.0, 170.0], [130.0, 170.0], [140.0, 170.0], [150.0, 170.0], [160.0, 170.0], [170.0, 170.0], [180.0, 170.0], [190.0, 170.0], [200.0, 170.0], [210.0, 170.0], [220.0, 170.0], [230.0, 170.0], [240.0, 170.0], [250.0, 170.0], [260.0, 170.0], [270.0, 170.0], [280.0, 170.0], [290.0, 170.0], [300.0, 170.0], [310.0, 170.0], [320.0, 170.0], [330.0, 170.0], [340.0, 180.0], [340.0, 190.0], [340.0, 200.0], [340.0, 210.0], [340.0, 220.0], [340.0, 230.0], [340.0, 240.0], [340.0, 250.0], [340.0, 260.0], [340.0, 270.0], [340.0, 280.0], [340.0, 290.0], [340.0, 300.0], [340.0, 310.0], [340.0, 320.0], [340.0, 330.0], [340.0, 340.0], [340.0, 350.0], [340.0, 360.0], [340.0, 370.0], [340.0, 380.0], [340.0, 390.0], [340.0, 400.0], [340.0, 410.0], [350.0, 420.0], [360.0, 430.0], [370.0, 440.0], [380.0, 440.0], [390.0, 440.0], [400.0, 440.0], [410.0, 440.0], [420.0, 450.0], [430.0, 460.0], [440.0, 470.0], [440.0, 480.0], [440.0, 490.0], [440.0, 500.0], [440.0, 510.0], [440.0, 520.0], [440.0, 530.0], [440.0, 540.0], [440.0, 550.0], [440.0, 560.0], [440.0, 570.0], [440.0, 580.0], [440.0, 590.0], [440.0, 600.0], [440.0, 610.0], [440.0, 620.0], [430.0, 630.0], [420.0, 630.0], [410.0, 630.0], [400.0, 630.0], [390.0, 630.0], [380.0, 630.0], [370.0, 630.0], [360.0, 630.0], [350.0, 630.0], [340.0, 630.0], [330.0, 630.0], [320.0, 630.0], [310.0, 630.0], [300.0, 630.0], [290.0, 630.0], [280.0, 630.0], [270.0, 620.0], [270.0, 610.0], [270.0, 600.0], [270.0, 590.0], [270.0, 580.0], [270.0, 570.0], [270.0, 560.0], [270.0, 550.0], [270.0, 540.0], [270.0, 530.0], [270.0, 520.0], [270.0, 510.0], [270.0, 500.0], [270.0, 490.0], [260.0, 480.0], [250.0, 470.0], [240.0, 460.0], [230.0, 450.0], [220.0, 440.0], [210.0, 440.0], [200.0, 440.0], [190.0, 440.0], [180.0, 440.0], [170.0, 430.0], [170.0, 420.0], [170.0, 410.0], [170.0, 400.0], [170.0, 390.0], [170.0, 380.0], [170.0, 370.0], [170.0, 360.0], [170.0, 350.0], [170.0, 340.0], [170.0, 330.0], [170.0, 320.0], [170.0, 310.0], [170.0, 300.0], [170.0, 290.0], [170.0, 280.0], [180.0, 270.0], [190.0, 270.0], [200.0, 270.0], [210.0, 270.0], [220.0, 270.0], [230.0, 270.0], [240.0, 270.0], [250.0, 280.0], [250.0, 290.0], [250.0, 300.0], [250.0, 310.0], [250.0, 320.0], [250.0, 330.0], [250.0, 340.0], [260.0, 350.0]
]

routePoints = np.array(routePoints)
routePoints /= 10
routePoints = routePoints.tolist()

In [66]:
def optimizeRoute(routePoints):
    opti = [routePoints[0]]
    for i in range(1, len(routePoints) -1 ):
        if routePoints[i-1][0] == routePoints[i][0] and routePoints[i][0] == routePoints[i+1][0]:
            pass
        elif routePoints[i-1][1] == routePoints[i][1] and routePoints[i][1] == routePoints[i+1][1]:
            pass
        else:
            opti.append(routePoints[i])
    opti.append(routePoints[-1])
    return opti

In [68]:

route = routePoints
# route = optimizeRoute(routePoints)
routex, routey = zip(*route)

# fig = plt.figure(figsize=(6, 6))
# ax = fig.add_subplot(1,1,1)
# planline, = ax.plot(routex, routey, label = "Planned")
# simline, = ax.plot([], [], label = "Simulated")
# # scat = ax.scatter([], [], s=2, c=[], cmap='plasma', label = "Simulated")
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.title('Route following')
# plt.legend()
# plt.grid(True)
# plt.axis('square')
# plt.show()

def update(yaw_kp, yaw_ki, yaw_kd, acc_kp, acc_ki, acc_kd, targetVel, L):
    model = FaithNew()
    time = 0
    resx, resy = [], []
    vel, yaw = 0, 0
    resacc = []
    state = [0,0,0,0]

    traj = Trajectory(route, L=L) # type: ignore
    goal = traj.getPoint(len(route) - 1)

    PIDAcc = PIDController(Kp=acc_kp, Ki=acc_ki, Kd=acc_kd)
    PIDYaw = PIDController(Kp=yaw_kp, Ki=yaw_ki, Kd=yaw_kd)

    if len(route) <= 1: return

    while getDistance([state[0], state[1]], goal) > 1:
        if time > T: break
        # print(getDistance([state[0], state[1]], goal))
        targetPoint = traj.getTargetPoint([state[0], state[1]])
        targetYaw = math.atan2(
                targetPoint[1] - state[1],
                targetPoint[0] - state[0]
            )
        targetYaw = math.degrees(targetYaw)
        # targetYaw = normalize(targetYaw, -180, 180)
        dYaw = PIDYaw.calculate(
            setpoint=targetYaw,
            feedback_value=state[2]
        )

        dAcc = PIDAcc.calculate(
            setpoint= targetVel,
            feedback_value= state[3] #/model.wb.maximumSpeed()
        )
        vel += dAcc*dt
        yaw += dYaw*dt
        speed_x = vel * math.sin(yaw)
        speed_y = vel * math.cos(yaw)

        model.wb.setLinearVelocity(dAcc)
        model.wb.setAngularVelocity(dYaw)
        model.update()

        # print(
        #     f"xy:({state[0]:.2f}, {state[1]:.2f}),\tdAcc: {dAcc:.2f},\tvel: {state[3]:.2f},\tdYaw: {dYaw:.2f},\th:{math.degrees(state[2]):.2f} deg,\tyaw_error:{PIDYaw.prev_error:.2f}"
        # )

        state = [
            model.pos[0],
            model.pos[1],
            model.h,
            math.hypot(model.vel[0], model.vel[1])
        ]        
        resx.append(model.pos[0])
        resy.append(model.pos[1])
        resacc.append(dAcc)
        time += dt
    # ax.clear()
    # ax.scatter(resx, resy, s=2, c=resacc, cmap='plasma', label = "Simulated")
    # fig.colorbar(resacc)
    # print(list(zip(resx, resy)))
    # simline.set_data(resx, resy)
    # fig.canvas.draw_idle()
    plt.figure(figsize=(6, 6))
    plt.plot(routex, routey, label = "Planned")
    plt.scatter(resx, resy, s=2, c=resacc, cmap='plasma', label = "Simulated")
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Route following')
    plt.legend()
    plt.grid(True)
    plt.axis('square')
    plt.colorbar()
    plt.show()

def simulation(yaw_kp, yaw_ki, yaw_kd, acc_kp, acc_ki, acc_kd, targetVel, L):
    global startedFollow
    global state
    model = Faith()
    time = 0
    resx, resy = [], []
    resacc = []
    state = [0,0,0,0]

    opti = optimizeRoute(routePoints)

    traj = Trajectory(opti, L=L)
    routex, routey = zip(*opti)
    goal = traj.getPoint(len(opti) - 1)
    # global PIDAcc
    # global PIDYaw
    PIDAcc = PIDController(Kp=acc_kp, Ki=acc_ki, Kd=acc_kd)
    PIDYaw = PIDController(Kp=yaw_kp, Ki=yaw_ki, Kd=yaw_kd)
    # PIDYaw.Kp = yaw_kp
    # PIDYaw.Ki = yaw_ki
    # PIDYaw.Kd = yaw_kd
    # PIDAcc.Kp = acc_kp
    # PIDAcc.Ki = acc_ki
    # PIDAcc.Kd = acc_kd
    # targetVel = tVel
    ## Check if route is plotted
    if len(opti) <= 1: 
        _output = [0.0, 0.0, 0.0]
        return

    ### Plotted: follow the route

    # model.wb.setMaxSpeed(600)

    # print(f"PID Acc Kp: {PIDAcc.Kp}, Ki: {PIDAcc.Ki}, Kd: {PIDAcc.Kd}")
    # print(f"PID Yaw Kp: {PIDYaw.Kp}, Ki: {PIDYaw.Ki}, Kd: {PIDYaw.Kd}")

    prev_yaw, prev_vel = 0, 0

    while time < T:
        while getDistance(pos(), goal) > 1:
            if time > T: break
            # print(getDistance([state[0], state[1]], goal))
            targetPoint = traj.getTargetPoint([state[0], state[1]])
            targetYaw = math.atan2(
                    targetPoint[1] - state[1],
                    targetPoint[0] - state[0]
                )
            targetYaw = math.degrees(targetYaw)
            targetYaw = normalize(targetYaw, -180, 180)
            
            dYaw = PIDYaw.calculate(
                setpoint=targetYaw,
                feedback_value=state[2]
            )

            dAcc = PIDAcc.calculate(
                setpoint= targetVel,
                feedback_value= state[3]#/model.wb.maximumSpeed()
            )
            # dYaw = normalize(dYaw, -math.pi, math.pi)
            # print((targetYaw, state[2], dYaw))

            # speed_x = dAcc * math.sin(dYaw) # output x
            # speed_y = dAcc * math.cos(dYaw) # output y
            # speed_x = math.sin(dYaw)
            # speed_y = math.cos(dYaw)

            # print(f"dAcc: {dAcc:.2f},\tvel: {state[3]:.2f},\tdYaw: {dYaw:.2f},\txy:({state[0]:.2f}), {state[1]:.2f},\th:{math.degrees(state[2]):.2f} deg,\tyaw_error:{PIDYaw.prev_error:.2f}")

            # if speed_x > 1.0: speed_x = 1.0
            # if speed_x < -1.0: speed_x = -1.0
            # if speed_y > 1.0: speed_y = 1.0
            # if speed_y < -1.0: speed_y = -1.0

            # _output = [speed_x, speed_y, 0.0]
            
            prev_vel += dAcc*dt
            prev_yaw += dYaw*dt

            speed_x = prev_vel * math.sin(prev_yaw)
            speed_y = prev_vel * math.cos(prev_yaw)
            # model.wb.enable()
            # model.wb.setMaxSpeed(800)
            # model.wb.inputXY(speed_x, speed_y)


            
            model.wb.setSpeed(1)
            # model.wb.setMaxSpeed(prev_vel)
            model.wb.setMaxSpeed(dAcc)
            # model.wb.setAngle(math.radians(270))
            # model.wb.setAngle(prev_yaw)
            # model.wb.setAngularVel(prev_yaw)
            model.wb.setAngularVel(dYaw)
            model.update()
            
            state = [
                 model.lastPosition[0],
                 model.lastPosition[1],
                 model.heading,
                 math.hypot(model.velocity[0], model.velocity[1])
                #  model.linvelocity
                ]
            # print(state)
            
            resx.append(model.lastPosition[0])
            resy.append(model.lastPosition[1])
            resacc.append(dAcc)
            time += dt
    # print(list(zip(resx, resy)))
    plt.figure(figsize=(6, 6))
    plt.plot(routex, routey, label = "Planned")
    plt.scatter(resx, resy, s=2, c=resacc, cmap='plasma', label = "Simulated")
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Route following')
    plt.legend()
    plt.grid(True)
    plt.axis('square')
    plt.colorbar()
    # ax.set_aspect('equal', adjustable='box')
    plt.show()

def changepid(yaw_kp=1.0, yaw_ki=0.01, yaw_kd=0.00, acc_kp=1.0, acc_ki=0.0, acc_kd=0.00, tVel = 0.2, Time=360.0, Lad=5):
    global PIDAcc
    global PIDYaw
    global targetVel
    global T
    # print('changepid')
    T = Time
    update(yaw_kp, yaw_ki, yaw_kd, acc_kp, acc_ki, acc_kd, tVel, Lad)

interact(
    changepid, 
    yaw_kp=(0.0, 5.0, 0.01), 
    yaw_ki=(0.0, 1.0, 0.01), 
    yaw_kd=(0.0, 1.0, 0.001), 
    acc_kp=(0.0, 5.0, 0.01), 
    acc_ki=(0.0, 1.0, 0.01), 
    acc_kd=(0.0, 1.0, 0.001),
    tVel=(0.0, 1.0, 0.01), 
    Time=(1.0, 360.0, 1.0), 
    Lad=(1,100,1)
);


interactive(children=(FloatSlider(value=1.0, description='yaw_kp', max=5.0, step=0.01), FloatSlider(value=0.01…